In [41]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds


In [42]:
# Version check
import google.protobuf

print("TF:", tf.__version__)
print("TFDS:", tfds.__version__)
print("protobuf:", google.protobuf.__version__)


TF: 2.12.0
TFDS: 4.9.2
protobuf: 3.20.3


In [48]:
# Dataset load
# Reload dataset with a proper 80/20 split
(train_ds, val_ds), info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],  # 80% train, 20% validation
    with_info=True,
    as_supervised=True
)


In [49]:
# Preprocessing
from tensorflow.keras.applications.efficientnet import preprocess_input

IMG_SIZE = 224   # standard input size for many pretrained models
BATCH_SIZE = 32  # how many images to feed the model at once

# Function to resize and normalize each image
def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)  # EfficientNet-specific normalization
    return image, label

# Apply preprocessing to the dataset and shuffle and batch the data
train_ds = train_ds.map(format_example).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = val_ds.map(format_example).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Visualization of sample images
# Take one batch from the dataset
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    
    for i in range(9):  # show 9 images
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())  # convert Tensor to NumPy for plotting
        plt.title(int(labels[i].numpy()))  # show class index as title
        plt.axis("off")


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import Sequential, layers

# Number of flower classes (from dataset info)
num_classes = info.features["label"].num_classes
print("Number of classes:", num_classes)

# Load EfficientNetB0 base model (pretrained on ImageNet)
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,   # exclude the original ImageNet classifier
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

# Freeze the base model (initial training phase)
base_model.trainable = False

# Build our custom classifier on top
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # reduce feature maps
    layers.Dropout(0.5),              # prevent overfitting
    layers.Dense(num_classes, activation="softmax")  # final classifier
])

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# Show model summary
model.summary()


In [50]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras import layers, Model

# Input layer
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Preprocessing
x = preprocess_input(inputs)

# Base model (pretrained)
base_model = EfficientNetB0(include_top=False, weights="imagenet")
base_model.trainable = False
x = base_model(x, training=False)

# Pooling + dropout + dense
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Build functional model
model = Model(inputs, outputs)

# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling2d_5   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_5 (Dropout)         (None, 1280)              0         
                                                                 
 dense_5 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 4,055,976
Trainable params: 6,405
Non-trainable params: 4,049,571
_____________________________________________

In [46]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        filepath="best_model.h5",
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]


In [51]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
92/92 [==============================] - ETA: 0s - loss: 1.5497 - accuracy: 0.3188
Epoch 1: val_accuracy improved from 0.21662 to 0.63079, saving model to best_model.h5
92/92 [==============================] - 103s 1s/step - loss: 1.5497 - accuracy: 0.3188 - val_loss: 1.1989 - val_accuracy: 0.6308
Epoch 2/10
92/92 [==============================] - ETA: 0s - loss: 1.1611 - accuracy: 0.5848
Epoch 2: val_accuracy improved from 0.63079 to 0.76703, saving model to best_model.h5
92/92 [==============================] - 96s 1s/step - loss: 1.1611 - accuracy: 0.5848 - val_loss: 0.9453 - val_accuracy: 0.7670
Epoch 3/10
92/92 [==============================] - ETA: 0s - loss: 0.9556 - accuracy: 0.6907
Epoch 3: val_accuracy improved from 0.76703 to 0.81063, saving model to best_model.h5
92/92 [==============================] - 95s 1s/step - loss: 0.9556 - accuracy: 0.6907 - val_loss: 0.7895 - val_accuracy: 0.8106
Epoch 4/10
92/92 [==============================] - ETA: 0s - loss: 0.83

In [ ]:
# Unfreeze all layers except BatchNorm (they can destabilize training)
# for layer in base_model.layers:
#     if not isinstance(layer, tf.keras.layers.BatchNormalization):
#         layer.trainable = True

# Unfreeze top 50 layers
for layer in base_model.layers[-50:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True


# Re-compile with a smaller learning rate for fine-tuning
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# Train again (fine-tuning)
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


In [ ]:
# Data augmentation
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
], name="data_augmentation")

# Rebuild model with augmentation
augmented_model = Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation="softmax")
])

# Compile again with a small learning rate for fine-tuning
augmented_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train again (longer this time)
history_aug = augmented_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)


In [ ]:
for images, labels in train_ds.take(1):
    print("Images batch shape:", images.shape)  # should be (32, 224, 224, 3)
    print("Labels batch shape:", labels.shape)  # should be (32,)
    print("Unique labels in this batch:", np.unique(labels.numpy()))


In [ ]:
small_train = train_ds.take(1)  # just 1 batch of 32 images

history_small = model.fit(
    small_train,
    epochs=20
)
